In [14]:
import ipfshttpclient
import sqlite3
import hashlib
import os
from tqdm import tqdm

def cid_to_int(cid):
    return int(hashlib.sha256(cid.encode()).hexdigest(), 16) % (2**31 - 1)

conn = sqlite3.connect('nft.db')
client = ipfshttpclient.connect(timeout=300)

In [15]:
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS base (
    id INTEGER PRIMARY KEY,
    hash TEXT NOT NULL
)
''')

In [28]:
cursor.execute("DELETE FROM base")
conn.commit()

In [16]:
azuki_path = "nft_images/azuki"
bayc_path = "nft_images/bayc"
cryptopunks_path = "nft_images/cryptopunks"

In [29]:
def register_db(image_path):
    image_list = os.listdir(image_path)
    for i in tqdm(range(len(image_list)), desc="Register DB"):
        test_img_path = os.path.join(image_path, image_list[i])
        test_img_hash = client.files.stat("/" + test_img_path)['Hash']
        test_img_id = cid_to_int(test_img_hash)
        cursor.execute("INSERT INTO base (id, hash) VALUES (?, ?)", (test_img_id, test_img_hash))
        conn.commit()

In [30]:
register_db(azuki_path)

Register DB: 100%|████████████████████████| 10000/10000 [04:18<00:00, 38.66it/s]


In [32]:
register_db(bayc_path)

Register DB: 100%|███████████████████████| 10000/10000 [01:39<00:00, 100.19it/s]


In [33]:
register_db(cryptopunks_path)

Register DB: 100%|████████████████████████| 10000/10000 [05:46<00:00, 28.84it/s]
